In [4]:
import asyncio
import websockets
import json
import datetime
import requests
import hashlib
import urllib.parse
import hmac
import time
import logging
import os
from collections import deque
import numpy as np

class BinanceAPI:
    def __init__(self, api_key: str, api_secret: str, testnet: bool = False):
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_url = "https://testnet.binancefuture.com" if testnet else "https://fapi.binance.com"

    def _generate_signature(self, params: dict) -> str:
        """Generate HMAC SHA256 signature for request."""
        query = urllib.parse.urlencode(params)
        return hmac.new(self.api_secret.encode(), query.encode(), hashlib.sha256).hexdigest()

    def _send_request(self, method: str, endpoint: str, params=None, signed=False) -> dict:
        """Send request to Binance API."""
        url = f"{self.base_url}{endpoint}"
        headers = {"X-MBX-APIKEY": self.api_key}
        if params is None:
            params = {}
        if signed:
            params["timestamp"] = int(time.time() * 1000)
            params["signature"] = self._generate_signature(params)
        try:
            if method == "GET":
                response = requests.get(url, headers=headers, params=params)
            elif method == "POST":
                response = requests.post(url, headers=headers, json=params)
            elif method == "DELETE":
                response = requests.delete(url, headers=headers, params=params)
            else:
                raise ValueError(f"Invalid method: {method}")
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            logging.error(f"API request failed: {e}")
            return None

    def get_mark_price(self, symbol: str) -> dict:
        """Get mark price for a symbol."""
        return self._send_request("GET", "/fapi/v1/premiumIndex", {"symbol": symbol})

    def get_position_risk(self, symbol: str = None) -> dict:
        """Get position risk information."""
        params = {"symbol": symbol} if symbol else {}
        return self._send_request("GET", "/fapi/v3/positionRisk", params, signed=True)

    def create_order(self, symbol: str, side: str, order_type: str, quantity: float, 
                    price: float = None, time_in_force: str = None, reduce_only: bool = False) -> dict:
        """Create a new order on Binance Futures."""
        params = {
            "symbol": symbol,
            "side": side,
            "type": order_type,
            "quantity": quantity,
            "reduceOnly": reduce_only,
        }
        if order_type == "LIMIT":
            if price is None or time_in_force is None:
                raise ValueError("Price and timeInForce are required for LIMIT orders.")
            params["price"] = price
            params["timeInForce"] = time_in_force
        return self._send_request("POST", "/fapi/v1/order", params, signed=True)

In [5]:
# Initialize Binance API
binance_api = BinanceAPI(api_key="TGZ6PvNeQc3c3ctlzm0UOdkgr1fi5oEMMPXDK9Dns51VXGKYGIirlOJ8de5TYNRC", 
                         api_secret="Ng4YmUDDzq7W9l5F08qcY3Qq2OXms4xE7A9nlslDIxP2agjVWqmZbOOxCRTZEHOl")

In [6]:
response = binance_api.create_order(
                    symbol="DOGEUSDT",
                    side="BUY",
                    order_type="MARKET",
                    quantity=40)  # Fixed quantity for 
print(response)

ERROR:root:API request failed: 400 Client Error: Bad Request for url: https://fapi.binance.com/fapi/v1/order


None
